You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
# import os
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [14]:
from langchain_openai import OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo-16k")

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [15]:
researcher = Agent(
    role="Researcher",
    goal="Given a block of HTML code from Google Scholar related to {topic}, "
         " extract relevant data.",
    backstory="You're working on summarizing the latest scholarly publications "
              "on the topic: {topic}. Provide the author, name, date of "
              "publication, and link to the PDF file in JSON format for all "
              "publications within the last {num_days} dats. Only provide this "
              "data for publications you can get all relevant data points for.\n"
              "Your work is the basis for the Summarizer.",
    llm=llm,
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [16]:
summarizer = Agent(
    role="Summarizer",
    goal="Craft a well-formatted email summarizing up to 5 scholarly "
         "publications about the topic: {topic}",
    backstory="You will write a regular digest to be delivered through email "
              "for readers interested in the topic: {topic}. You base your "
              "writing on the work of the Researcher, who provides details in "
              "JSON format about the most recent publications about the topic. "
              "Make sure to include the name of the publication, author, date "
              "published, and provide a link to the PDF. Include a short "
              "summary of the content of each publication.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
# editor = Agent(
#     role="Editor",
#     goal="Edit a given email draft to align with "
#          "the writing style of the organization. ",
#     backstory="You are an editor who receives a blog post "
#               "from the Content Writer. "
#               "Your goal is to review the blog post "
#               "to ensure that it follows journalistic best practices,"
#               "provides balanced viewpoints "
#               "when providing opinions or assertions, "
#               "and also avoids major controversial topics "
#               "or opinions when possible.",
#     allow_delegation=False,
#     verbose=True
# )

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [17]:
research = Task(
    description=(
        "1. Extract the author, name, date, and link to PDF for all publications "
            "from the provided html object: {html}.\n"
        "2. Remove any publications without all of the required pieces of data.\n"
        "3. Remove any publications written more than {num_days} days ago.\n"
    ),
    expected_output="A JSON-formatted document.",
    agent=researcher,
)

### Task: Write

In [18]:
summarize = Task(
    description=(
        "1. Use the JSON document to draft an email summarizing the latest "
            "publications on {topic}.\n"
        "2. Include a summary of each publication's findings.\n"
		"3. Limit content to up to 5 of the most interesting publications.\n"
        "4. Sign off the email with a salutation from Ainesh Pandey."
        "5. Proofread for grammatical errors\n"
    ),
    expected_output="A well-written email draft, with a numbered list of up to "
        "5 publications, summarizing the latest scholarly research on {topic}.",
    agent=summarizer,
)

### Task: Edit

In [9]:
# edit = Task(
#     description=("Proofread the given blog post for "
#                  "grammatical errors and "
#                  "alignment with the brand's voice."),
#     expected_output="A well-written blog post in markdown format, "
#                     "ready for publication, "
#                     "each section should have 2 or 3 paragraphs.",
#     agent=editor
# )

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [21]:
crew = Crew(
    agents=[researcher, summarizer],
    tasks=[research, summarize],
    verbose=2
)

2024-05-27 13:45:43,471 - 8706336000 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [22]:
from utils import get_html

topic = "gun violence"
result = crew.kickoff(
    inputs={
        "topic": topic,
        "num_days": 7,
        "html": get_html(topic)
        })

 [DEBUG]: == Working Agent: Researcher
 [INFO]: == Starting Task: 1. Extract the author, name, date, and link to PDF for all publications from the provided html object: [<div class="gs_r gs_or gs_scl" data-aid="HcHJTvM0n7QJ" data-cid="HcHJTvM0n7QJ" data-did="HcHJTvM0n7QJ" data-lid="" data-rp="0"><div class="gs_ggs gs_fl"><div class="gs_ggsd"><div class="gs_or_ggsm" ontouchstart="gs_evt_dsp(event)" tabindex="-1"><a data-clk="hl=en&amp;sa=T&amp;oi=gga&amp;ct=gga&amp;cd=0&amp;d=13015179667727565085&amp;ei=LBtVZrqpCI3A6rQP3vaVoAs" data-clk-atid="HcHJTvM0n7QJ" href="https://escholarship.org/content/qt386222r1/qt386222r1_noSplash_485922152f7b5da04e470599d4d3856b.pdf"><span class="gs_ctg2">[PDF]</span> escholarship.org</a></div></div></div><div class="gs_ri"><h3 class="gs_rt" ontouchstart="gs_evt_dsp(event)"><a data-clk="hl=en&amp;sa=T&amp;ct=res&amp;cd=0&amp;d=13015179667727565085&amp;ei=LBtVZrqpCI3A6rQP3vaVoAs" data-clk-atid="HcHJTvM0n7QJ" href="https://escholarship.org/uc/item/386222r1" id

2024-05-27 13:45:49,268 - 8706336000 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-3.5-16k to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-27 13:45:49,277 - 8706336000 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-3.5-16k to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-27 13:45:49,296 - 8706336000 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-3.5-16k to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.')
2024-05-27 13:45:49,298 - 8706336000 - manager.py-manager:282 - WARNING: Error in TokenCalcHandler.on_llm_start callback: KeyError('Could not automatically map gpt-3.5-16k to a tokeniser. Pleas

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-3.5-16k` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

- Display the results of your execution as markdown in the notebook.

In [ ]:
from IPython.display import Markdown
Markdown(result)

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).